# 附一 基于 LangChain 自定义 LLM

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

在本部分，我们以百度文心大模型为例，讲述如何基于 LangChain 自定义 LLM，让我们基于 LangChain 搭建的应用能够支持百度文心、讯飞星火等国内大模型。

本部分涉及相对更多 LangChain、大模型调用的技术细节，有精力同学可以学习部署，如无精力可以直接使用后续代码来支持调用。

要实现自定义 LLM，需要定义一个自定义类继承自 LangChain 的 LLM 基类，然后定义两个函数：  
① _call 方法，其接受一个字符串，并返回一个字符串，即模型的核心调用；  
② _identifying_params 方法，用于打印 LLM 信息。  

首先我们导入所需的第三方库：

In [12]:
import json
import time
from typing import Any, List, Mapping, Optional, Dict, Union, Tuple
import requests
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.utils import get_from_dict_or_env
from pydantic import Field, model_validator

由于百度文心使用双重秘钥进行认证，用户需要先基于 API_Key 与 Secret_Key 来获取 access_token，再使用 access_token 来实现对模型的调用（详见《3. 调用百度文心》），因此我们需要先定义一个 get_access_token 方法来获取 access_token：

In [4]:
def get_access_token(api_key : str, secret_key : str):
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """
    # 指定网址
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"
    # 设置 POST 访问
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    # 通过 POST 访问获取账户对应的 access_token
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")

接着我们定义一个继承自 LLM 类的自定义 LLM 类：

In [5]:
# 继承自 langchain.llms.base.LLM
class Wenxin_LLM(LLM):
    # 原生接口地址
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant"
    # 默认选用 ERNIE-Bot-turbo 模型，即目前一般所说的百度文心大模型
    model_name: str = Field(default="ERNIE-Bot-turbo", alias="model")
    # 访问时延上限
    request_timeout: Optional[Union[float, Tuple[float, float]]] = None
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = None
    # Secret_Key
    secret_key : str = None
    # access_token
    access_token: str = None
    # 必备的可选参数
    model_kwargs: Dict[str, Any] = Field(default_factory=dict)


上述初始化涵盖了我们平时常用的参数，也可以根据实际需求与文心的 API 加入更多的参数。

接下来我们实现一个初始化方法 init_access_token，当模型的 access_token 为空时调用：

In [6]:
def init_access_token(self):
    if self.api_key != None and self.secret_key != None:
        # 两个 Key 均非空才可以获取 access_token
        try:
            self.access_token = get_access_token(self.api_key, self.secret_key)
        except Exception as e:
            print(e)
            print("获取 access_token 失败，请检查 Key")
    else:
        print("API_Key 或 Secret_Key 为空，请检查 Key")

接下来我们实现核心的方法——调用模型 API：

In [7]:
def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
    # 除 prompt 参数外，其他参数并没有被用到，但当我们通过 LangChain 调用时会传入这些参数，因此必须设置
    # 如果 access_token 为空，初始化 access_token
    if self.access_token == None:
        self.init_access_token()
    # API 调用 url
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token={}".format(self.access_token)
    # 配置 POST 参数
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ],
        'temperature' : self.temperature
    })
    headers = {
        'Content-Type': 'application/json'
    }
    # 发起请求
    response = requests.request("POST", url, headers=headers, data=payload, timeout=self.request_timeout)
    if response.status_code == 200:
        # 返回的是一个 Json 字符串
        js = json.loads(response.text)
        return js["result"]
    else:
        return "请求失败"

然后我们还需要定义一下模型的描述方法：

In [9]:
# 首先定义一个返回默认参数的方法
@property
def _default_params(self) -> Dict[str, Any]:
    """获取调用Ennie API的默认参数。"""
    normal_params = {
        "temperature": self.temperature,
        "request_timeout": self.request_timeout,
        }
    return {**normal_params}


@property
def _identifying_params(self) -> Mapping[str, Any]:
    """Get the identifying parameters."""
    return {**{"model_name": self.model_name}, **self._default_params}

通过上述步骤，我们就可以基于 LangChain 定义百度文心的调用方式了。我们将此代码封装在 wenxin_llm.py 文件中，将在讲述如何调用百度文心的 Notebook 中直接使用该 LLM。